# Import Section

In [0]:
# Path
import os

# Data
import numpy as np

# Image
from PIL import Image # used in image processing function

# Tensorflow
import tensorflow as tf

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

# For Tensorflow Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.initializers import RandomNormal

# Google Drive
from google.colab import drive


# Colab Function

In [0]:
def gpu_check():
  """
      Check GPU RAM status. Since google colab share gpu resource amount of its
      user, you want to make sure there are enough GPU RAM that are free to use.
      Recommend at least 3000MB free GPU RAM.
  """
  # memory footprint support libraries/code
  !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
  !pip install gputil
  !pip install psutil
  !pip install humanize
  import psutil
  import humanize
  import os
  import GPUtil as GPU
  GPUs = GPU.getGPUs()
  # XXX: only one GPU on Colab and isn’t guaranteed
  gpu = GPUs[0]
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

In [0]:
def mount_drive():
  """
      Mount your google drive to google colab, so that you can access the
      dataset via google drive.
      
      YOUR DIR PATH SHOULD BE: gdrive/"Colab Notebooks"
  """
  drive.mount('/content/gdrive')
  

# Config

In [0]:
"""
    config project in this cell.
"""

# Env, where you running this project
# Google colab: 'colab'
env = 'colab'


In [13]:
""" 
     _______________________________________________
    | YOU SHOULD NOT CHANGE ANY THING IN THIS CELL. |
    |_______________________________________________|
"""

# config dictionary
config = {
    'colab':{
        'data_dir':'/content/gdrive/My Drive/Colab Notebooks/data',
        'checkpoint_dir':'/content/gdrive/My Drive/Colab Notebooks/checkpoints',
        'log_dir':'/content/gdrive/My Drive/Colab Notebooks/logs',
        'weight_dir':'/content/gdrive/My Drive/Colab Notebooks/weights'
    }
}

# setting all variables
data_dir = config[env]['data_dir']
checkpoint_dir = config[env]['checkpoint_dir']
log_dir = config[env]['log_dir']
weight_dir = config[env]['weight_dir']

if env == 'colab':
  # mount google dirve
  mount_drive()
  # check GPU
  gpu_check()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Gen RAM Free: 12.3 GB  | Proc size: 1.5 GB
GPU RAM Free: 11311MB | Used: 130MB | Util   1% | Total 11441MB


# Image Processing Function

In [0]:
def array_to_img(x, mode='YCbCr'):
  """
      Convert array to image using YCbCr color
      
      Args:
        x: array of image.
        mode: channel mode, default to 'YCbCr'
              1 (1-bit pixels, black and white, stored with one pixel per byte)
              L (8-bit pixels, black and white)
              P (8-bit pixels, mapped to any other mode using a color palette)
              RGB (3x8-bit pixels, true color)
              RGBA (4x8-bit pixels, true color with transparency mask)
              CMYK (4x8-bit pixels, color separation)
              YCbCr (3x8-bit pixels, color video format)
              Note that this refers to the JPEG, and not the ITU-R BT.2020, standard
              LAB (3x8-bit pixels, the L*a*b color space)
              HSV (3x8-bit pixels, Hue, Saturation, Value color space)
              I (32-bit signed integer pixels)
              F (32-bit floating point pixels)
  """
  return Image.fromarray(x.astype('uint8'), mode=mode)


def bicubic_rescale(image, scale):
  """
      Rescale image using bicubic interpolation.
      
      Args:
        image: image
        scale: use integer for up scaling. use 1/integer for down scaling
  """
  # make sure scale is valid
  if isinstance(scale, (float, int)):
    size = (np.array(image.size) * scale).astype(int)
  '''
  WARNING
  image.resize might lead to image displacement
  https://hackernoon.com/how-tensorflows-tf-image-resize-stole-60-days-of-my-life-aba5eb093f35
  switch to tf.image.resize_bicubic
  '''
  return image.resize(size, resample=Image.BICUBIC)


def modcrop(image, scale):
  """
      To scale down the original image, there must be no remainder while scaling
      operation.
      
      All we want to do in here is to subtract the remainder from height and 
      width of original image size, and cut the original image to the new size.
      
      Args:
        image: original image
        scale: must be int
  """
  if not isinstance(scale, int):
    raise Exception('utils.modcrop: scale must be int')
  size = np.array(image.size)
  size -= size % scale
  return image.crop([0, 0, *size])

# Data Preprocessing

In [0]:
def load_image_pair(path, scale=3, greyscale=False, same_size=False):
  """
      Down scaling a hight resolution image to a low resolution image and
      return both of them.
      
      Args:
        path: image path
        scale: scale of down scaling, must be a int
        greyscale: return only Y channel
  """
  image = load_img(path)
  image = image.convert('YCbCr')
  
  if greyscale:
    Y, Cb, Cr = image.split()
    Y.show()
    image = Y
  
  hr_image = modcrop(image, scale)
  lr_image = bicubic_rescale(hr_image, 1 / scale)
  lr_image = bicubic_rescale(lr_image, scale)
  # TODO: blur a sub-image by a proper Gaussian kernel
  return lr_image, hr_image


def generate_sub_images(image, size, stride):
  """
      Cut image into sub images.
      
      Args:
        image: image
        size: size of sub image
        stride: distance of how much the window shifts by in each of the 
                dimensions
  """
  for i in range(0, image.size[0] - size + 1, stride):
        for j in range(0, image.size[1] - size + 1, stride):
            # yield return a generator, or a list of number
            yield image.crop([i, j, i + size, j + size])


def load_set(dataset_name, lr_sub_size, lr_sub_stride, scale, same_size=False, greyscale=False):
  """
      Load all image from a directory and cut them into small sub image.
      
      Args:
        dataset_name: name of dir of the data set
        lr_sub_size: low resolution sub image size
        lr_sub_stride: stride when crop sub image
        scale: down scale value
        same_size: hr, lr have the same size
        greyscale: return only Y channel
  """
  if not all(isinstance(i, int) for i in [lr_sub_size, lr_sub_stride, scale]):
    raise Exception('utils.load_set: lr_sub_size, stride, scale must be int')
    
  # compute parameters for hight resolution image
  hr_sub_size = lr_sub_size * scale
  hr_sub_stride = lr_sub_stride * scale

  lr_sub_arrays = []
  hr_sub_arrays = []
  for file_name in os.listdir(os.path.join(data_dir, dataset_name)):
    path = os.path.join(data_dir, dataset_name, file_name)
    lr_image, hr_image = load_image_pair(str(path), scale=scale, greyscale=greyscale, same_size=same_size)
    lr_sub_arrays += [img_to_array(img) for img in generate_sub_images(lr_image, size=lr_sub_size, stride=lr_sub_stride)]
    hr_sub_arrays += [img_to_array(img) for img in generate_sub_images(hr_image, size=hr_sub_size, stride=hr_sub_stride)]
  
  # convert list to np.array
  x = np.stack(lr_sub_arrays)
  y = np.stack(hr_sub_arrays)
  
  return x, y


# TODO
# normalization?
# Seng: add a layer called BatchNormalization in model
# https://keras.io/layers/normalization/

# Helper Function

In [0]:
def train(
    model,
    train_set,
    val_set,
    epochs=1,
    steps_per_epoch=30,
    validation_steps=3,
    resume=True):
  """
    train function for all model.
    
    
  """
  # define callbacks
  callbacks = [
    # Save checkpoints of model at regular intervals
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_dir,
        save_best_only=True
    ),
      
    # Interrupt training if `val_loss` stops improving for over 2 epochs
    tf.keras.callbacks.EarlyStopping(
        patience=2,
        monitor='val_loss'
    ),
      
    # Write TensorBoard logs to `./logs` directory
    tf.keras.callbacks.TensorBoard(
        log_dir=log_dir
    )
  ]
  
  # inherit weights
  if resume:
    # get model name
    name = ''
    if isinstance(model, type(srcnn)):
      name = 'srcnn'
      
    model.load_weights(os.path.join(weight_dir,name))
  
  # Train
  model.fit(train_set, epochs=epochs, steps_per_epoch=steps_per_epoch,
            validation_data=val_set, validation_steps=validation_steps)
  
  # TODO: plot metrics
  

def test(model, test_set, steps=30, metrics=None):
  # test
  model.evaluate(test_set, steps=steps)

# Model

In [0]:
def srcnn(img_size, channel=1, f1=9, n1=64, n2=32, f3=5):
  '''
  img_size: the size of the input image
  channel: the number of channels in the input image
  f1: the size of filter (must be odd #)
  n1: the number of filter apply on layer 1
  n2: the number of filter apply on layer 2
  f3: the size of filter (must be odd #)
  
  
  from: https://arxiv.org/abs/1501.00092
  '''  
  
  if not isinstance(img_size, (int)):
    raise Exception('img_size is not a valid size in srcnn model')
  
  initializer = RandomNormal(mean=0.0, stddev=0.001)
  model = Sequential()
  model.add(Conv2D(filters=n1, kernel_size=f1, padding='same', activation='relu',
                   kernel_initializer=initializer, input_shape=(img_size,img_size,channel)))
  model.add(Conv2D(filters=n2, kernel_size=1, padding='same', activation='relu',
                  kernel_initializer=initializer))
  model.add(Conv2D(filters=channel, kernel_size=f3, padding='same', activation='linear',
                  kernel_initializer=initializer))
  
  # either SGD or Adam
  optimizer = SGD(lr=0.0003)
  # optimizer = Adam(lr=0.0003)
  model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_squared_error'])
  
  model.summary()
  
  return model



# Model Training

## Model Config

In [16]:
# size of sub image
size = 32

# strde when crop image
stride = 14

# upscaling factor
scale = 1

# batch size
batch = 32

# size of low resolution and high resolution image is the same?
same_size = True

# greyscale ON or OFF? if ON, img will only contain Y channel
greyscale = True

# whcih training dataset you want to use?
train_dataset_dir = '91-image'

# which validation dataset you want to use?
val_dataset_dir = 'Set5'

# which testing dataset you want to use?
test_dataset_dir = 'Set14'

# which model you want to train or test?
model = srcnn(size, 1, 9, 64, 32, 5)

# epochs
epochs = 15000

# steps_per_epoch
steps_per_epoch = 30

# validation_steps
validation_steps = 3

# resume last traning?
resume = False
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 64)        5248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 32)        2080      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 1)         801       
Total params: 8,129
Trainable params: 8,129
Non-trainable params: 0
_________________________________________________________________


## Make Data

In [0]:
""" 
     _______________________________________________
    | YOU SHOULD NOT CHANGE ANY THING IN THIS CELL. |
    |_______________________________________________|
"""

# load tranning dataset
train_lr, train_hr = load_set(train_dataset_dir, size, stride, scale, same_size, greyscale)

# make tf.data dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_lr, train_hr))
train_dataset = train_dataset.batch(batch)
train_dataset = train_dataset.repeat()

# load validation dataset
val_lr, val_hr = load_set(val_dataset_dir, size, stride, scale, same_size, greyscale)
# make tf.data dataset
val_dataset = tf.data.Dataset.from_tensor_slices((val_lr, val_hr))
val_dataset = val_dataset.batch(batch//10)
val_dataset = val_dataset.repeat()

In [18]:
# Train
train(model, train_dataset, val_dataset, epochs, steps_per_epoch, validation_steps, resume)

Instructions for updating:
Use tf.cast instead.
Epoch 1/15000
30/30 [==============================] - 2s 83ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/15000
30/30 [==============================] - 0s 9ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/15000
30/30 [==============================] - 0s 9ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/15000
30/30 [==============================] - 0s 9ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/15000
30/30 [==============================] - 0s 9ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/15000
30/30 [==============================] - 0s 9ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/15000
30/30 [========================

KeyboardInterrupt: ignored